# Jayveersinh Raj
# BS-DS-01
# j.raj@innopolis.university

# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 40 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set
- 20 points - Evaluate on Github Typo Corpus (for masters only)


Remarks: 
- Use Python 3 or greater
- Max is 80 points for bachelors, 100 points for masters

## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html).

[N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf)

You may also wnat to implement:
- spell-checking for a concrete language - Russian, Tatar, Ukranian, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

Important - your project should not be a mere code copy-paste from somewhere. Implement yourself, analyze why it was suggested this way, and think of improvements/customization.

Your solution should be able to perform 4 corrections per second (3-5 words in an example) on a typical cpu.

## Justify your decisions
In your implementation you will need to decide which ngram dataset to use, which weights to assign for edit1, edit2 or absent words probabilities, beam search parameters and etc. Write down justificaitons for these choices.

## Spell checker

In [9]:
# The read bigrams function to read the bigrams
def read_file(file_path):
    freq_table = {}
    with open(file_path, 'r', encoding='ISO-8859-1') as f:
        for line in f:         
            freq, *bigrams = line.strip().split('\t') 
            bigrams = ' '.join(bigrams)
            freq_table[bigrams.lower()] = int(freq)
    return freq_table

BIGRAMS = read_file('/content/w2_.txt')

# Total frequency
N=sum(BIGRAMS.values())

def P(bigram):
    "Probability of `bigram`."
    return BIGRAMS.get(bigram, 0) / N

def edits1(word):
    '''
    "All edits that are one edit away from `word`."
    word : word
    return : a collection of all editable strings (words and non-words alike) that can be created with a single edit
    '''

     # Define ASCII codes for lowercase letters
    alphabets = range(97, 123)

    # Split the word into left and right substrings
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]

    # Remove one letter from the word
    deletes = [left + right[1:] for left, right in splits if right]

    # Transpose adjacent letters
    transposes = [left + right[1] + right[0] + right[2:] for left, right in splits if len(right) > 1]

    # Replace a letter with another letter from alphabets
    replaces = [left + chr(c) + right[1:] for left, right in splits if right for c in alphabets if c != ord(right[0])]

    # Insert a letter from alphabets
    inserts = [left + chr(c) + right for left, right in splits for c in alphabets]

    # Return all possible words as a set
    return set(deletes + transposes + replaces + inserts)


def edits2(word): 
    '''
    All edits that are two edits away from `word`.
    word : word
    return : a collection of all editable strings (words and non-words alike) that can be created with a two edit
    '''

    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


def existing(words): 
    '''
    The subset of `words` that appear in the dictionary of WORDS.
    word : word
    return : a set of all existing words in the dictionary
    '''

    return set(w for w in words if w in BIGRAMS.keys())

def candidates(word): 
    '''
    Generate possible spelling corrections for word.
    word : word
    return : a collection of viable candidates
    '''
    return (existing([word]) or existing(edits1(word)) or existing(edits2(word)) or [word])

def correction(word): 
    '''
    Most probable spelling correction for word.
    word: word
    returns the most likely
    '''
    return max(candidates(word), key=P)

## Testing on a word by calling `correction function`

In [33]:
correction('a cicle')

'a circle'

## Justification
### 1. Choosing bi-gram over n-gram with n=5 : 
An n-gram is a continuous series of n elements from the text, whereas a bigram is a pair of adjacent words in a passage (e.g., letters, words, or even characters). Bigrams and n-grams are used in spell checkers to determine how to spell a word correctly based on the context in which it appears.

Bigrams are typically preferred for spell checking over n-grams with high n values (such as n=5) for a number of reasons:
- `Smaller Context`: Bigrams give the spelling correction algorithm a more focused context. Using a bigram, the algorithm simply needs to take into account the target word's two closest neighbors. It is frequently possible to determine the right spelling from this context. However, n-grams with a large n number may supply too much context, making it more challenging to extract the important details.
- `Improved Handling of Word Variations`: The algorithm is better able to manage word usage changes when using bigrams. For example, if the target word is "their", the bigram "their house" provides more relevant context than a 5-gram that includes numerous words between "their" and "house". Despite the fact that "their" might not be the most popular option in that location, this can help the algorithm identify that it is right.

- `Computationally Efficient`: Bigrams are typically computed more quickly and with fewer resources than n-grams with a high value of n. This is because the algorithm only needs to look at a little bit of information for each bigram, which results in fewer bigrams to take into account.

Generally, `n-grams` can offer `more context` than `bigrams`, but they can also be `harder` to deal with and `more expensive to compute`. `Bigrams` provide a `reasonable balance` between `context and efficiency` for many `applications`, including `spell checking`.

### 2. The core spell checking algorithm:
It is very similar to `Norvig's Solution`.
- `The probability function`: It is simply `getting it from dictionary`, and dividing its `frequency` by `total frequencies`. It could be done `more complex`, and `even better`, but it would create more `complexity`. For example `Hamming distance` and `Levenshtein distance` could be used. But `Frequency table` is already given which `eases` the `complexity`.

- `The edits function`: They are same as Norvig's solution, the reason is that in order to `fairly compare` both uni-gram (i.e. Norvig's solution), and bi-gram as I created, they must be the same. Moreover, the point down to this would explain it even better

- `Possible customizations considered`: 
1. Swapping vowels: Adds an extra layer of complexity, and would not help much

2. Using `AI` based `algorithms` like `Genetic Algorithm`: Computationally very heavy, best solution to provide approximate better solutions using a very efficient `fitting function`, but `computationally inefficient`, `memory inefficient`

3. `Conclusion`: All in all there exists better solutions, but for this one to be in parallel with Norvig's solution or to compare it with his solution, it should be fair

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

## Generating a small test set to compare with Norvig's solution

In [22]:
# Creating a test set 
test_lines = [
"a bicyle: a bycyle",
"a circle: a cicle",
"an umbrella: an umbrlela",
"a baby: a babi",
"during sports: dring sports",
"as above: as aboev",
"a another: a anoter",
"dying species: dying species",
"during lunch: dring lunch",
]

# Writing to a test_data.txt file
with open("test_data.txt", "w") as f:
  for pairs in test_lines:
    f.write(f"{pairs}\n")

## Evaluation function

In [31]:
import time
def test_bigram_spellchecker():
    '''Test the bigram-based spell checker on the above generated test data'''
    from collections import Counter
    
    # load test data
    with open('/content/test_data.txt') as f:
        test_data = f.read().splitlines()
    
    # storing total number of test set
    n = len(test_data)

    # start timer
    start = time.process_time()
    
    # iterate over test data and evaluate spell checker performance
    total_count, correct_count = 0, 0
    for line in test_data:
        words = line.split(':')
        correct_word = words[0]
        misspelled_words = words[1:]
        
        # get the prediction on the misspelled word
        for word in misspelled_words:
           suggestion = correction(word)
           if suggestion == correct_word:
             correct_count += 1
           total_count += 1
                    
    dt = time.process_time() - start
    # print results
    accuracy = correct_count / total_count
    print(f'Test results: {correct_count}/{total_count} correct ({accuracy:.2%} accuracy), with {int(n/dt)} words per second')

### Calling the evaluation function to compare

In [32]:
test_bigram_spellchecker()

Test results: 5/9 correct (55.56% accuracy), with 4 words per second


## The Comparison

## The above implemented bi-grams solution : `~56%`
## Norvig's solution : `68%`

## Evaluate on Github Typo Corpus
Now, you need to evaluate your solution on the Github Typo Corpus. Don't forget to compare the accuracy with the Norvig's solution.

In [ ]:
!wget https://github-typo-corpus.s3.amazonaws.com/data/github-typo-corpus.v1.0.0.jsonl.gz
!gzip -d github-typo-corpus.v1.0.0.jsonl.gz

In [ ]:
import jsonlines

dataset_file = "github-typo-corpus.v1.0.0.jsonl"

dataset = []
other_langs = set()

with jsonlines.open(dataset_file) as reader:
    for obj in reader:
        for edit in obj['edits']:
            if edit['src']['lang'] == 'eng' and edit['is_typo']:
                src, tgt = edit['src']['text'], edit['tgt']['text']
                if src.lower() != tgt.lower():
                    dataset.append((src, tgt))
                
print(f"Dataset size = {len(dataset)}")

Please, explore the dataset. You may see, that this is
- mostly markdown
- some common mistakes with do/does
- some just refer to punctuation typos (which we do not consider)

In [ ]:
for pair in dataset[1010:1020]:
    print(f"{pair[0]} => {pair[1]}")

Compare your implementation with Norvig's solution on this dataset

In [ ]:
limit = 10000
counter = limit
for i, (src, target) in enumerate(dataset):
    if i == limit:
        break
    # YOUR CODE HERE